In [12]:
import flywheel
import datetime
import logging

fw = flywheel.Client()
project_id = '6009d38842d76c8da2cca869'
gear = fw.lookup('gears/wmhssysu2debug')
date = datetime.datetime.now()
date_str = "{}-{}-{}_{}:{}".format(date.year, date.month, date.day, date.hour, date.minute)
date_str

'2021-9-20_16:32'

In [13]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [14]:
# There are some older sessions in this project, but we want the 20 we just uploaded (conveniently, all the names begin with 20)
sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id) if x.label.startswith('20')]
len(sessions)

20

In [15]:
for session in sessions:
    
    t1_img = None
    flair_img = None
    flair_10x_img = None
    
    for acq in session.acquisitions():
        # determine the input files
        for f in acq.files:
            if 'reorient_T1w' in f.name:
                t1_img = f
            elif 'reorient_flair' in f.name:
                flair_img = f
            elif 'reorient_10_flair' in f.name:
                flair_10x_img = f
            else:
                continue
                
        if t1_img and flair_img:
            # Run the gear with the normal FLAIR
            inputs = {'FLAIR': flair_img, 'T1': t1_img}
            label = 'sysu2_{}_test_WT'.format(date_str)
            config = {}
            analysis_id = gear.run(analysis_label=label, config=config, inputs=inputs, destination=session)
            log.info("Running {} with {} and {}.".format(label, flair_img, t1_img))

        if t1_img and flair_10x_img:
            # Run the gear with the 10x FLAIR
            inputs = {'FLAIR': flair_10x_img, 'T1': t1_img}
            label = 'sysu2_{}_10x_test_WT'.format(date_str)
            config = {}
            analysis_id = gear.run(analysis_label=label, config=config, inputs=inputs, destination=session)
            log.info("Running {} with {} and {}.".format(label, flair_10x_img, t1_img))
